<a href="https://colab.research.google.com/github/softmurata/generative-ai-handsbook/blob/main/investigation/tripabroad/tripabroad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation

In [ ]:
!pip install -q elevenlabs

Import Libraries

In [2]:
from elevenlabs import generate
from IPython.display import Audio

Create Audio Test

In [ ]:
sf = 44100

audio = generate(
  text="Yes, please remove your shoes and place them in the bin provided.",
  voice="Bella",
  model="eleven_multilingual_v2"
)

Audio(audio,rate=sf)

change voice settings

In [19]:
from elevenlabs import Voice, VoiceSettings, generate, save

audio = generate(
    text="Hello! My name is Bella.",
    voice=Voice(
        voice_id='EXAVITQu4vr4xnSDxMaL',
        settings=VoiceSettings(stability=0.8, similarity_boost=0.5, style=0.5, use_speaker_boost=True)
    )
)

Audio(audio,rate=sf)
save(audio, "output.mp3")

In [ ]:
# slow output
!ffmpeg -i output.mp3 -vf setpts=PTS*2 -af atempo=0.5 output_slow.mp3

In [ ]:
Audio("/content/output_slow.mp3",rate=sf)

Create Audio scripts

In [ ]:
from elevenlabs import voices
voices = voices()
voices[0]

In [7]:
# Docs.mdのDataPreparationを参照
# chatgptから会話内容を作成して、それぞれにまとめる
Y_list = [
    "Good evening. Could you recommend any specialties of the house?",
    "That sounds great. I'll go with the seafood paella, please. And may I have a glass of red wine to go with it?",
    "No, that should be it for now, thank you.",
    "The paella was delicious, thank you. Could you please bring the check?",
    "No, everything was perfect, thank you. By the way, are there any interesting places around here that you would recommend visiting?"
]

W_list = [
    "Of course! Our chef's specials tonight are the seafood paella and the grilled sirloin steak. They are both highly recommended.",
    "Excellent choice! A glass of red wine with the seafood paella it is. Anything else for you?",
     "How was your meal? Is there anything else I can get for you?",
    "Of course. I'll bring it right away.",
    "Certainly! If you're interested in local history, there's a museum just a few blocks away, and the waterfront promenade is also quite beautiful in the evening.",
    "You're welcome. Enjoy the rest of your evening!"
]

# ToDo: 年齢とか、性別とかそういう要素で判定できるようにする
Y_voice = "Clyde"
W_voice = "Rachel"

In [8]:
audios = []

for yl, wl in zip(Y_list, W_list):

  y_audio = generate(
    text=yl,
    voice=Y_voice,
    model="eleven_multilingual_v2"
  )
  w_audio = generate(
    text=wl,
    voice=W_voice,
    model="eleven_multilingual_v2"
  )

  audios.append(y_audio)
  audios.append(w_audio)

In [ ]:
sf = 44100
all_audio = audios[0]
for a in audios[1:]:
  all_audio += a
Audio(all_audio,rate=sf)

Create audio with slow speed

In [24]:
# Authentication周りについては以下
# https://docs.elevenlabs.io/api-reference/quick-start/authentication
from elevenlabs import set_api_key
set_api_key("58fac52259018f41bacc6030970ca620")

In [ ]:
audios = []

tempo = 0.8

for idx, (yl, wl) in enumerate(zip(Y_list, W_list)):

  y_audio = generate(
    text=yl,
    voice=Y_voice,
    model="eleven_multilingual_v2"
  )
  w_audio = generate(
    text=wl,
    voice=W_voice,
    model="eleven_multilingual_v2"
  )

  y_audio_path = f"y_{idx}.mp3"
  y_slow_audio_path = f"y_{idx}_slow.mp3"
  save(y_audio, y_audio_path)

  # slow output
  !ffmpeg -i $y_audio_path -vf setpts=PTS*2 -af atempo=$tempo $y_slow_audio_path

  audios.append(y_slow_audio_path)

  w_audio_path = f"w_{idx}.mp3"
  w_slow_audio_path = f"w_{idx}_slow.mp3"
  save(w_audio, w_audio_path)

  # slow output
  !ffmpeg -i $w_audio_path -vf setpts=PTS*2 -af atempo=$tempo $w_slow_audio_path

  audios.append(w_slow_audio_path)

Concatenate audio

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
 # enter the path of your audio file

sounds = AudioSegment.from_mp3(audios[0])

for a in audios[1:]:
  target_sound = AudioSegment.from_mp3(a)
  sounds += target_sound

sounds.export("output.wav", format="wav")

display(Audio("output.wav"))